In [2]:
import os
import rasterio

import numpy as np

from rasterio.windows import Window

# Windowing

In [3]:
def get_windows(img_dim, patch_size=(240, 240), stride=(240, 240)):
    patch_size = np.array(patch_size)
    stride = np.array(stride)
    img_dim = np.array(img_dim)
    
    max_dim = (img_dim//patch_size)*patch_size - patch_size

    ys = np.arange(0, img_dim[0], stride[0])
    xs = np.arange(0, img_dim[1], stride[1])

    tlc = np.array(np.meshgrid(ys, xs)).T.reshape(-1, 2)
    tlc = tlc[tlc[:, 0] <= max_dim[0]]
    tlc = tlc[tlc[:, 1] <= max_dim[1]]
    
    windows = []
    for y,x in tlc.astype(int):
        windows.append(Window(x, y, patch_size[1], patch_size[0]))

    return windows

In [50]:
path = "/Users/zwarshavsky/Downloads/1000_full_congo_export_v12_all_bands_Feb_11_12_22_04_2021.tif"
out_path = "."
patch_size = 100
stride = 100
SAVE = False
SAVE_INDIVIDUAL = False

raster = rasterio.open(path)
src = raster.read()
windows = get_windows(raster.shape, (patch_size, patch_size), (stride, stride))

for j, window in enumerate(windows):
    
    data = raster.read((1,3,5),window=window, masked=True)
    
    print(j, window, data.shape)

#     if np.any(data.mask):
        # THE PATCH CONTAINS NODATA PIXELS, HANDLE AS NEEDED

#     data = DO_PREPROCESSING(data.data)
#     result = DO_INFERENCE(data)

    if SAVE:

        if SAVE_INDIVIDUAL:
            meta = raster.meta.copy()
            # Get the window specific transform - IF we want to save windows independantly
    #         trans = raster.window_transform(window)
            meta.update({
    #                 'transform': trans,
                    'dtype': src.dtype
                })

            with rasterio.open(f"{out_path}/some_chip_{j}.tif", 'w', **meta) as dest:
                dest.write(data)
        else:
            meta = raster.meta.copy()

            with rasterio.open(f"{out_path}/some_chip_{j}.tif", 'w+', **meta) as dest:
                dest.write(data, window=window)

0 Window(col_off=0, row_off=0, width=100, height=100) (3, 100, 100)
1 Window(col_off=100, row_off=0, width=100, height=100) (3, 100, 100)
2 Window(col_off=200, row_off=0, width=100, height=100) (3, 100, 100)
3 Window(col_off=300, row_off=0, width=100, height=100) (3, 100, 100)
4 Window(col_off=400, row_off=0, width=100, height=100) (3, 100, 100)
5 Window(col_off=500, row_off=0, width=100, height=100) (3, 100, 100)
6 Window(col_off=600, row_off=0, width=100, height=100) (3, 100, 100)
7 Window(col_off=700, row_off=0, width=100, height=100) (3, 100, 100)
8 Window(col_off=800, row_off=0, width=100, height=100) (3, 100, 100)
9 Window(col_off=900, row_off=0, width=100, height=100) (3, 100, 100)
10 Window(col_off=1000, row_off=0, width=100, height=100) (3, 100, 100)
11 Window(col_off=1100, row_off=0, width=100, height=100) (3, 100, 100)
12 Window(col_off=1200, row_off=0, width=100, height=100) (3, 100, 100)
13 Window(col_off=1300, row_off=0, width=100, height=100) (3, 100, 100)
14 Window(col_

In [32]:
src.dtype

dtype('uint16')